# 模型训练之前，进行的batch操作
### 1、tensorflow版本的dataset

### 2、pytorch版本的dataset

In [1]:
import tensorflow as tf
import torch

### tokenier/vocab

In [3]:
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]'
UNKNOWN_TOKEN = '[UNK]'
START_DECODING = '[START]'
STOP_DECODING = '[STOP]'

class Vocab:
    def __init__(self,vocab_path,max_size):
        self.word2id={UNKNOWN_TOKEN:0,PAD_TOKEN:1,START_DECODING:2,STOP_DECODING:3}
        self.id2word={0:UNKNOWN_TOKEN,1:PAD_TOKEN,2:START_DECODING,3:STOP_DECODING}
        self.count=4
        
        with open(vocab_path,"r",encoding="utf-8") as f:
            for line in f:
                pair=line.split()
                if len(pair)!=2:
                    print("Warning:the error line is :{}"%line)
                w=pair[0]
                if w in [SENTENCE_START,SENTENCE_END,PAD_TOKEN,UNKNOWN_TOKEN,START_DECODING,STOP_DECODING]:
                    raise Exception("word <%s> is exception"%w)
                if w in self.word2id:
                    raise Exception("word <%s> is repetition")
                
                if max_size>=self.count:
                    self.word2id[w]=self.count
                    self.id2word[self.count]=w
                    self.count+=1
                else:
                    break
        
    def word_to_id(self,word):
        if w not in self.word2id:
            return self.word2id[UNKNOWN_TOKEN]
        return self.word2id[word]
    
    def id_to_word(self,w_id):
        if w_id not in self.id2word:
            raise Exception("id %s can not found"%w_id)
        return self.id2word[w_id]
    
    def size(self):
        return self.count

### 1、tf版本1
#### 读取数据：
    * tf.data.TextLineDataset(txt_path)----->将一句输入 整合 指定格式输出  
    * tf.data.Dataset.zip((dataset_train_x, dataset_train_y))   #当作zip使用
    * train_dataset.shuffle(1000, reshuffle_each_iteration=True).repeat()   #shuffle+repeat()，buffer_size=1000
#### 将迭代器处理成dataset:
    * tf.data.Dataset.from_generator(generator,output_types={key:tf.int32....},output_shape={key:shape...})   #generator必须是迭代器
    为什么要再用这个？
    答：因为上面读取数据后，还要进行train、test、eval不同状况下的处理，返回的数据不是dataset类型了，而是字典。。。。
#### 构建batch数据,并设置默认填充：
    * dataset.padded_batch(batch_size,padded_shapes={key:shape...},padding_values={key:value...},drop_reminder=true)
    drop_reminder:表示最后一个不满足的batch是否丢弃
#### map操作：
    * 对dataset的所有输出进行格式或者映射(map)的处理